<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_4"><div id="image_img" class="header_image_4"></div></td>
        <td class="header_text">EMG Sensor - Unit Conversion </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">pre-process&#9729;emg&#9729;conversion</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

The <strong><span class="color2">OpenSignals</span></strong> outputted file formats contain raw data, so each sample has a digital unit.

In scientific terms it is recommended the use of specific units, like electric tension (V) or electric current (A).
Each sensor that <strong>PLUX</strong> commercialise has a datasheet where a transfer function is mentioned for unit conversion be done.

The next lines are intended to explain how this conversion can be programmatically done.

In spite of the unit conversion procedure has some common steps applicable to all sensors, the current <strong><span class="color5">Jupyter Notebook</span></strong> is dedicated to the unit conversion procedure of signals acquired with EMG sensor.

<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [19]:
# biosignalsnotebooks Python package with useful functions that support and complement the available Notebooks
import biosignalsnotebooks as bsnb

# Function used for creating a numpy array, where a mathematical operation can be applied to each entry 
# in an easy and automatic way. On the other side, linspace, here will be used for generation of a time-axis.
from numpy import array, linspace

<p class="steps">3 - Extraction of the transfer function from the beginning of the second page</p> 

\begin{equation}
EMG_{mV} = \frac{(\frac{ADC}{2^n} - \frac{1}{2}).VCC}{G_{EMG}}
\end{equation}

<table width="100%">
    <tr>
        <td width="33%" style="text-align:left;vertical-align:top">$EMG_{mV}$ - EMG value in mV</td>
        <td width="33%" style="text-align:left;vertical-align:top">$ADC$ - Digital value sampled from the channel (initialism of "Analog to Digital Converter")</td>
        <td width="33%" style="text-align:left;vertical-align:top">$n$ - Number of bits of the channel (dependent on the chosen resolution specified on <span class="color2">OpenSignals</span> previously to the acquisition stage [8, 12 or 16 bits])</td>
    </tr>
    <tr>
        <td style="text-align:left;vertical-align:top">$VCC$ - 3000 mV</td>
        <td style="text-align:left;vertical-align:top">$G_{EMG}$ - 1000</td>
        <td></td>
    </tr>
</table>

<p class="steps">4 - Loading of data stored in <strong><span class="color2">biosignalsnotebooks</span></strong> own signal library</p> 

In [20]:
# Data loading
data, header = bsnb.load("../../data/EMG_ana.h5", get_header=True)

print ("\033[1mHeader:\n\033[0m" + str(header) + "\n\033[1mData:\033[0m\n" + str(data))

Header:
{'channels': array([1]), 'comments': '', 'date': '2024-2-28', 'device': 'bitalino_rev', 'device connection': 'BTH0C:43:14:1C:2A:25', 'device name': '0C:43:14:1C:2A:25', 'digital IO': array([0, 0, 1, 1]), 'firmware version': 1282, 'resolution': array([ 4,  1,  1,  1,  1, 10]), 'sampling rate': 1000, 'sync interval': 2, 'time': '11:12:10.193', 'sensor': ['EMGBITREV'], 'column labels': {1: 'channel_1'}}
Data:
{'CH1': array([507, 509, 508, ..., 509, 507, 508], dtype=uint32)}


In the following cell, some relevant information is stored inside variables. This relevant information includes the mac-address of the device, channel number and signal acquisition parameters such as resolution and sampling rate.

For a detailed explanation of how to access this info, the <a href="../Load/signal_loading_preparatory_steps.ipynb" target="_blank">"Signal Loading - Working with File Header"<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> Notebook should be consulted.

In [21]:
device = header["device name"]
print(f'Device: {device}')

ch = list(data.keys())[0]
print(f'Channel: {ch}')

sr = header["sampling rate"]
print(f'Sampling rate: {sr}')

resolution = header["resolution"][-1]
print(f'Resolution: {resolution}')

Device: 0C:43:14:1C:2A:25
Channel: CH1
Sampling rate: 1000
Resolution: 10


Access to acquired signal samples and storage inside a new variable.

In [22]:
signal = data[ch]

<p class="steps">5 - Final unit conversion (to <span class="color5">mV</span>) by applying the transfer function sample by sample</p>
Definition of $VCC$ and $G_{EMG}$ constants

In [23]:
vcc = 3300 # mV
gain = 1000

In [24]:
signal_mv = (((array(signal) / 2**resolution) - 0.5) * vcc) / gain

<p class="steps">6 - Time axis generation</p>

In [25]:
time = bsnb.generate_time(signal_mv, sr)

Comparison between RAW and mV signal.

In [27]:
bsnb.plot([time, time], [signal, signal_mv], y_axis_label=["Raw Data", "Electric Voltage (mV)"], grid_lines=1, grid_columns=2, grid_plot=True)

<span class="color2">Similar Notebooks</span>, dedicated to the unit conversion of other sensors, are available in the following "conversion" section of <a href="../MainFiles/by_tag.ipynb">Notebooks Grouped by Tag page <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>

A list of PLUX&#39;s available sensors (and the respective datasheets) can be accessed at <a href="http://www.biosignalsplux.com/en/products/sensors">Sensors page <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>.

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 